In [9]:
!pip install youtube_transcript_api

In [10]:
import requests # api calls
import json # CONVERT TO JSON
import polars as pl # faster version of pandas
from google.colab import userdata # load secreats

from youtube_transcript_api import YouTubeTranscriptApi

In [11]:
import os
my_key = userdata.get('my_key')

In [12]:
def getVideoRecords(response: requests.models.Response) -> list:
    """
        Function to extract YouTube video data from GET request response
    """

    video_record_list = []

    for raw_item in json.loads(response.text)['items']:

        # only execute for youtube videos
        if raw_item['id']['kind'] != "youtube#video":
            continue

        video_record = {}
        video_record['video_id'] = raw_item['id']['videoId']
        video_record['datetime'] = raw_item['snippet']['publishedAt']
        video_record['title'] = raw_item['snippet']['title']

        video_record_list.append(video_record)

    return video_record_list

In [13]:
# define channel ID
channel_id = 'UCa9gErQ9AE5jT2DZLjXBIdA'

# define url for API
url = 'https://www.googleapis.com/youtube/v3/search'

# initialize page token
page_token = None

# intialize list to store video data
video_record_list = []

In [14]:
%%time
# extract video data across multiple search result pages
while page_token != 0:
    # define parameters for API call
    # max result is reduced to 2. because it exceed the daily limit.
    params = {"key": my_key, 'channelId': channel_id, 'part': ["snippet","id"], 'order': "date", 'maxResults':2, 'pageToken': page_token}
    # make get request
    response = requests.get(url, params=params)

    # Check if the request was successful and the response contains 'items'
    if response.status_code == 200 and 'items' in json.loads(response.text):
        # append video records to list
        video_record_list += getVideoRecords(response)

        try:
            # grab next page token
            page_token = json.loads(response.text)['nextPageToken']
        except KeyError:
            # if no next page token kill while loop
            page_token = 0
    else:
        # If the request was not successful or 'items' is missing, stop the loop
        print(f"API request failed with status code {response.status_code} or no items found.")
        print(response.text) # Print the response text for debugging
        page_token = 0

API request failed with status code 403 or no items found.
{
  "error": {
    "code": 403,
    "message": "The request cannot be completed because you have exceeded your \u003ca href=\"/youtube/v3/getting-started#quota\"\u003equota\u003c/a\u003e.",
    "errors": [
      {
        "message": "The request cannot be completed because you have exceeded your \u003ca href=\"/youtube/v3/getting-started#quota\"\u003equota\u003c/a\u003e.",
        "domain": "youtube.quota",
        "reason": "quotaExceeded"
      }
    ]
  }
}

CPU times: user 5.63 ms, sys: 918 µs, total: 6.55 ms
Wall time: 72.4 ms


In [16]:
# write data to file
os.makedirs('data')
pl.DataFrame(video_record_list).write_parquet('data/video-ids.parquet')
pl.DataFrame(video_record_list).write_csv('data/video-ids.csv')

In [17]:
video_record_list

[]